In [19]:
import numpy as np
from typing import Tuple, List
import matplotlib.pylab as plt
import tidy3d as td
import tidy3d.web as web

freq0 = 2e14
freqs = [freq0]
wavelength = td.C_0 / freq0
bck_eps = 1.0**2
slab_eps0 = [4.0, 3.0, 2.0, 4.0]
slab_ds0 = [1.4, 2.1, 1.5, 4.25]
theta = np.pi/5
dl = 0.005

def make_sim(slab_eps=slab_eps0, slab_ds=slab_ds0) -> td.Simulation:

    # frequency setup
    wavelength = td.C_0 / freq0
    fwidth = freq0 / 10.0
    freqs = [freq0]

    # geometry setup
    bck_medium = td.Medium(permittivity=bck_eps)

    space_above = 2
    space_below = 2

    length_x = 0.2
    center_x = 0.0
    length_z = space_below + sum(slab_ds0) + space_above
    sim_size = (length_x, length_x, length_z)

    # make structures
    slabs = []
    z_start = -length_z/2 + space_below
    for (d, eps) in zip(slab_ds, slab_eps):
        slab = td.Structure(
            geometry=td.Box(center=[0, 0, z_start + d / 2], size=[td.inf, td.inf, d]),
            medium=td.Medium(permittivity=eps),
        )
        slabs.append(slab)
        z_start += d

    # source setup
    gaussian = td.GaussianPulse(freq0=freq0, fwidth=fwidth)
    src_z = -length_z / 2 + 3 * space_below / 4

    source = td.PlaneWave(
        center=(center_x, 0, src_z),
        size=(td.inf, td.inf, 0),
        source_time=gaussian,
        direction="+",
        angle_theta=theta,
        angle_phi=0,
        pol_angle=0,
    )

    # boundaries
    boundary_x = td.Boundary.bloch_from_source(
        source=source, domain_size=sim_size[0], axis=0, medium=bck_medium
    )
    boundary_spec = td.BoundarySpec(x=boundary_x, y=td.Boundary.periodic(), z=td.Boundary.pml(num_layers=40))

    # monitors
    mnt_z = length_z / 2 - wavelength
    monitor_1 = td.DiffractionMonitor(
        center=[0.0, 0.0, mnt_z],
        size=[td.inf, td.inf, 0],
        freqs=freqs,
        name="diffraction",
        normal_dir="+",
    )

    # make simulation
    return td.Simulation(
        size=sim_size,
        grid_spec=td.GridSpec.uniform(dl=dl),
        # structures=slabs,
        sources=[source],
        # monitors=[monitor_1],
        run_time=50 / fwidth,
        boundary_spec=boundary_spec,
        medium=bck_medium,
        subpixel=True,
        shutoff=1e-8,
    )

sim = make_sim()
sim_data = web.run(sim, task_name='error')

🚶  Starting 'error'...

[09:36:11] ERROR    Error running task!                                                                   ]8;id=650294;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/docs/source/notebooks/../../../tidy3d/tidy3d/log.py\log.py]8;;\:]8;id=215988;file:///Users/twhughes/Documents/Flexcompute/tidy3d-docs/docs/source/notebooks/../../../tidy3d/tidy3d/log.py#33\33]8;;\

WebError: (WebError(...), 'Error running task!')

In [ ]:
sim = make_sim()
f, ax = plt.subplots(1, 1, figsize=(10, 10))
sim.plot(y=0, ax=ax)
plt.show()

In [ ]:
sim_data = web.run(sim, task_name='error')